In [2]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

# Run the installation commands
!pip install opendatasets
!pip install ultralytics
!pip install pytubefix

# Clean up the notebook
clean_notebook()



Notebook cleaned.


In [4]:

import ultralytics
ultralytics.checks()

Ultralytics 8.3.47 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-80GB MIG 7g.80gb, 81051MiB)
Setup complete ✅ (256 CPUs, 2015.7 GB RAM, 288.1/3519.1 GB disk)


# Yolo 11 with Video files

In [ ]:
import cv2
from ultralytics import YOLO
from IPython.display import display, Image, clear_output
import numpy as np

# Set the desired width for resizing frames
set_width = 800

# Load the YOLO model
model = YOLO("yolo11n-seg.pt")

# Class names (ensure these match your model's training dataset, e.g., COCO)
class_names = model.names  # Assuming `model.names` provides the class name mapping

# Initialize video capture (0 for webcam, or provide a video file path)
video_path = ".././Videos/video4.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video source: {video_path}")
    cap.release()
    raise RuntimeError("Video initialization failed.")

try:
    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            print("End of video or failed to grab frame.")
            break
        
        # Resize the frame for faster processing
        ratio = set_width / frame.shape[1]
        frame = cv2.resize(frame, (set_width, int(frame.shape[0] * ratio)))

        # Convert BGR frame to RGB for YOLO inference
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Run YOLO inference on the frame
        results = model(rgb_frame, imgsz=640, verbose=False)

        # Draw segmentation masks and bounding boxes if results are available
        if len(results) > 0 and hasattr(results[0], 'masks'):
            masks = results[0].masks
            object_count = len(masks.data)

            # Create a combined mask initialized to zeros
            height, width = frame.shape[:2]
            combined_mask = np.zeros((height, width), dtype=np.uint8)

            for i in range(object_count):
                mask_array = masks.data[i].cpu().numpy().astype(np.uint8) * 255

                # Resize mask to match the frame if needed
                if mask_array.shape[:2] != (height, width):
                    mask_array = cv2.resize(mask_array, (width, height), interpolation=cv2.INTER_NEAREST)

                # Combine masks
                combined_mask = cv2.bitwise_or(combined_mask, mask_array)

                # Draw the mask on the frame
                frame[mask_array > 0] = [0, 255, 0]  # Example: green mask overlay

                # Get bounding box and label information
                if hasattr(masks, 'boxes'):
                    for box in masks.boxes:
                        # Extract bounding box coordinates
                        x1, y1, x2, y2 = map(int, box.xyxy[0])

                        # Draw rectangle
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                        # Get class ID and confidence
                        class_id = int(box.cls[0]) if hasattr(box.cls, '__iter__') else int(box.cls)
                        confidence = float(box.conf[0]) if hasattr(box.conf, '__iter__') else float(box.conf)

                        # Get class name
                        class_name = class_names[class_id] if class_id < len(class_names) else f"ID {class_id}"

                        # Create label text
                        label = f"{class_name}: {confidence:.2f}"
                        cv2.putText(frame, label, (x1, y1 - 10), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Encode the frame as JPEG
        _, buffer = cv2.imencode('.jpg', frame)
        img_bytes = buffer.tobytes()

        # Display the frame inline in the notebook
        display(Image(data=img_bytes))
        clear_output(wait=True)
finally:
    cap.release()
